# Text Generator

We are going to develop a text generator based on Markov Chains, and try different inputs and hiperparameters in order to achieve concrete outcomes that help us understand the possibilities of this technique. 

First we are going to model the problem with some classes in order to make the development easier.

In [ ]:
from numpy.random import choice

class Word:
    def __init__(self, word):
        self.word = word
        self.count_following_words = 0
        self.following_words = {}
        
    def add_following_word(other_word):
        self.following_words[other_word] = self.following_words.get(other_word, 0) + 1
        self.count_following_words += 1